In [6]:
# load libraries and scripts
#using PyPlot
using Plots
#using PyCall
using LaTeXStrings
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")
include("../scripts/get_params.jl")
include("../scripts/stable.jl")
include("../scripts/plotting.jl")
include("../scripts/dynamics.jl")
include("../scripts/disturbances.jl")
include("../scripts/ps_analysis.jl");

In [2]:
# load borders
border, scale_factor = import_border("../data/borders/border.json")

dx = 0.005
mesh = get_grid(border, dx); # renamed to be able to use the grid function for plotting

In [15]:
# compute or load grid parameters
contmod = get_params(mesh, scale_factor, "../data/disc/pantagruel.h5", Niter=10,
    dmax = 2*dx, patch=1900.0, bmin=52.0, tau=5.0e-7);

  3.530088 seconds (3.69 M allocations: 16.978 GiB, 9.12% gc time, 3.17% compilation time)
  3.061791 seconds (4.23 M allocations: 1.248 GiB, 13.32% gc time, 57.16% compilation time)


In [16]:
# get stable solution
compute_stable_sol!(contmod, interval = 10000, Niter = 250000, tol = 1e-7);
dm = load_discrete_model("../data/disc/pantagruel.h5", scale_factor);
th_disc = get_discrete_values(contmod.coord[contmod.isgrid,:], dm.coord, contmod.th[contmod.isgrid]);

[10000.0 0.00010416100217836011]
[20000.0 3.791699982347296e-5]
[30000.0 1.9966827604189774e-5]
[40000.0 1.6065244865082207e-5]
[50000.0 1.3633178183591887e-5]
[60000.0 1.2124254233136789e-5]
[70000.0 1.1183033745254534e-5]
[80000.0 1.0592072499981953e-5]
[90000.0 1.0219012503975833e-5]
[100000.0 9.982583263568046e-6]
[110000.0 9.832346266280823e-6]
[120000.0 9.736714051689077e-6]
[130000.0 9.675772984385489e-6]
[140000.0 9.636911797117342e-6]
[150000.0 9.612119969193422e-6]
[160000.0 9.596299616165815e-6]
[170000.0 9.586202564415558e-6]
[180000.0 9.579757661248856e-6]
[190000.0 9.575643655890254e-6]
[200000.0 9.573017446751209e-6]
[210000.0 9.571340945480067e-6]
[220000.0 9.5702706978118e-6]
[230000.0 9.569587464342e-6]
[240000.0 9.569151294241962e-6]
[250000.0 9.568872847753873e-6]
 28.570366 seconds (3.67 M allocations: 81.760 GiB, 7.41% gc time, 1.55% compilation time)


In [30]:
dm = load_discrete_model("../data/disc/pantagruel.h5", scale_factor);

vmin = min(minimum(dm.th), minimum(contmod.th))
vmax = max(maximum(dm.th), maximum(contmod.th))
borders = [border]
coord = dm.coord


temp = copy(dm.th)
temp .-= vmin
temp ./= (vmax - vmin)

    C(g::ColorGradient) = RGB[g[z] for z=temp]
    g = :inferno
    plot() # here to "clear" the figure
    for k in 1:length(borders)
        plot!(borders[k][:, 1], borders[k][:, 2], color=:black, linewidth=3.0)
    end
    scatter!(coord[:,2], coord[:,1], color=(cgrad(g) |> C), legend=false,
        markerstrokecolor=(cgrad(g) |> C), grid=false,
        showaxis=:hide, xaxis=nothing, yaxis=nothing, markersize=6.0,
        aspect_ratio=:equal)
savefig("../figures/steady1.svg")

In [31]:
    
temp = copy(contmod.th)
temp[.!contmod.isinside] .= NaN
c = :inferno
heatmap(contmod.yrange, contmod.xrange,
    reshape(temp, contmod.Ny, contmod.Nx),
    c = c, clim = (vmin, vmax))
    for k in 1:length(borders)
        p2 = plot!(borders[k][:, 1], borders[k][:, 2], color=:black,linewidth=3.0)
    end
    plot!(legend=false, grid=false, showaxis=:hide, xaxis=nothing,
    yaxis=nothing, aspect_ratio=:equal,colorbar=false)

savefig("../figures/steady2.svg")

In [33]:
scatter(dm.th, th_disc)
th_min = minimum(dm.th)
th_max = maximum(dm.th)
plot!([th_min; th_max], [th_min; th_max],color=:red, lw = 2, legend=false,
    size=(400,400), grid=false, aspect_ratio=:equal, ymirror=true)
savefig("../figures/steady3.svg")
